In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [2]:
# Load Raw Data
df_raw = pd.read_csv('../data/raw/ahmedabad_real_estate_data.csv')

print(f"📁 Dataset loaded: {df_raw.shape[0]} rows × {df_raw.shape[1]} columns")
print("\n📋 Column Names:")
print(df_raw.columns.tolist())
print("\n🔍 First 3 rows:")
df_raw.head(3)

📁 Dataset loaded: 2989 rows × 11 columns

📋 Column Names:
['Property Title', 'Price', 'Area', 'BHK', 'Bathrooms', 'Furnishing', 'Seller Type', 'Locality', 'City', 'Source', 'Raw_Details']

🔍 First 3 rows:


,Property Title,Price,Area,BHK,Bathrooms,Furnishing,Seller Type,Locality,City,Source,Raw_Details
0,"4, 5 BHK Flat,Duplexesin Shilaj",BIBequest Infraprojects Llp1/30Zero BrokerageS...,NaN,5 BHK,NaN,NaN,Agent,Ahmedabad,Ahmedabad,Housing.com,BI | Bequest Infraprojects Llp | 1 | / | 30 | ...
1,"3, 4 BHK Flatsin Vastral",PIPropus Infracon1/9Zero BrokeragePropus Clara...,NaN,4 BHK,NaN,NaN,Agent,Ahmedabad,Ahmedabad,Housing.com,PI | Propus Infracon | 1 | / | 9 | Zero Broker...
2,3 BHK Flatin Ognaj,VRVibgyor Reality1/14Zero BrokerageSculptureRE...,NaN,3 BHK,NaN,NaN,Agent,Ahmedabad,Ahmedabad,Housing.com,VR | Vibgyor Reality | 1 | / | 14 | Zero Broke...


In [3]:
# Data Quality Assessment
print("="*60)
print("DATA QUALITY REPORT")
print("="*60)

print("\n1️⃣ MISSING VALUES:")
missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct.round(2)
})
print(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False))

print("\n2️⃣ DATA TYPES:")
print(df_raw.dtypes)

print("\n3️⃣ DUPLICATE ROWS:")
duplicates = df_raw.duplicated().sum()
print(f"Total duplicates: {duplicates}")

DATA QUALITY REPORT

1️⃣ MISSING VALUES:
             Missing Count  Percentage
Bathrooms             2911       97.39
Furnishing              55        1.84
Area                    30        1.00
BHK                     15        0.50
Seller Type              2        0.07
Price                    1        0.03

2️⃣ DATA TYPES:
Property Title     object
Price              object
Area               object
BHK                object
Bathrooms         float64
Furnishing         object
Seller Type        object
Locality           object
City               object
Source             object
Raw_Details        object
dtype: object

3️⃣ DUPLICATE ROWS:
Total duplicates: 0


## 🧹 Step 1: Remove Duplicates

In [4]:
# Remove exact duplicates
df = df_raw.copy()
initial_count = len(df)

df = df.drop_duplicates()
after_exact = len(df)

# Remove duplicates based on key columns
df = df.drop_duplicates(subset=['Property Title', 'Price', 'Locality'], keep='first')
final_count = len(df)

print(f"🗑️ Removed {initial_count - final_count} duplicate records")
print(f"   - Exact duplicates: {initial_count - after_exact}")
print(f"   - Similar properties: {after_exact - final_count}")
print(f"✅ Remaining records: {final_count}")

🗑️ Removed 184 duplicate records
   - Exact duplicates: 0
   - Similar properties: 184
✅ Remaining records: 2805


## 🔧 Step 2: Clean Price Column

In [5]:
def clean_price(price_str):
    """Convert price string to numeric value in Lakhs"""
    if pd.isna(price_str) or price_str == 'N/A':
        return np.nan
    
    price_str = str(price_str).upper().strip()
    
    # Remove special characters except numbers, dots, and letters
    price_str = re.sub(r'[^0-9\.LCRA]', '', price_str)
    
    # Extract number
    numbers = re.findall(r'\d+\.?\d*', price_str)
    if not numbers:
        return np.nan
    
    value = float(numbers[0])
    
    # Convert to Lakhs
    if 'CR' in price_str or 'C' in price_str:
        return value * 100  # Crore to Lakh
    elif 'L' in price_str:
        return value
    else:
        # Assume Lakhs if no unit specified
        return value if value < 1000 else value / 100

# Apply cleaning
df['Price_Lakhs'] = df['Price'].apply(clean_price)

print("💰 Price Cleaning Results:")
print(f"   Valid prices: {df['Price_Lakhs'].notna().sum()}")
print(f"   Missing/Invalid: {df['Price_Lakhs'].isna().sum()}")
print(f"\n📊 Price Statistics (in Lakhs):")
print(df['Price_Lakhs'].describe())

# Sample conversions
print("\n🔍 Sample Price Conversions:")
print(df[['Price', 'Price_Lakhs']].dropna().head(10))

💰 Price Cleaning Results:
   Valid prices: 2804
   Missing/Invalid: 1

📊 Price Statistics (in Lakhs):
count      2804.000000
mean       4268.424394
std        4531.239162
min         100.000000
25%         413.750000
50%        4500.000000
75%        6500.000000
max      131300.000000
Name: Price_Lakhs, dtype: float64

🔍 Sample Price Conversions:
                                               Price  Price_Lakhs
0  BIBequest Infraprojects Llp1/30Zero BrokerageS...      13000.0
1  PIPropus Infracon1/9Zero BrokeragePropus Clara...       1900.0
2  VRVibgyor Reality1/14Zero BrokerageSculptureRE...      11400.0
3  Addor Group+2more1/10Zero BrokerageAddor Evara...        200.0
4  JWJanaadhar Western Projects LLP1/28Zero Broke...      12800.0
5  SHIVALIK GROUP1/13Zero BrokerageShivalik The C...      11300.0
6  Unique Infraspace Pvt Ltd+1more1/43Zero Broker...        100.0
7  Satya Sankalp Group+1more1/263D viewZero Broke...        100.0
8  Swara Group1/13Zero BrokerageSwara SkyparkRERA...     

## 📐 Step 3: Clean Area Column

In [6]:
def clean_area(area_str):
    """Convert area string to numeric value in square feet"""
    if pd.isna(area_str) or area_str == 'N/A':
        return np.nan
    
    area_str = str(area_str).lower().strip()
    
    # Extract numbers
    numbers = re.findall(r'\d+\.?\d*', area_str)
    if not numbers:
        return np.nan
    
    value = float(numbers[0])
    
    # Convert to square feet
    if 'sqyd' in area_str or 'sq.yd' in area_str or 'yard' in area_str:
        return value * 9  # sq yard to sq feet
    elif 'sqft' in area_str or 'sq.ft' in area_str or 'sqft' in area_str:
        return value
    elif 'sq. ft' in area_str or 'sq ft' in area_str:
        return value
    else:
        # Assume square feet if no unit
        return value if value > 100 else value * 100

# Apply cleaning
df['Area_SqFt'] = df['Area'].apply(clean_area)

print("📐 Area Cleaning Results:")
print(f"   Valid areas: {df['Area_SqFt'].notna().sum()}")
print(f"   Missing/Invalid: {df['Area_SqFt'].isna().sum()}")
print(f"\n📊 Area Statistics (in Sq.Ft):")
print(df['Area_SqFt'].describe())

# Sample conversions
print("\n🔍 Sample Area Conversions:")
print(df[['Area', 'Area_SqFt']].dropna().head(10))

📐 Area Cleaning Results:
   Valid areas: 2772
   Missing/Invalid: 33

📊 Area Statistics (in Sq.Ft):
count     2772.000000
mean      1557.220058
std       1856.009990
min         11.000000
25%        629.750000
50%       1050.000000
75%       1635.000000
max      10000.000000
Name: Area_SqFt, dtype: float64

🔍 Sample Area Conversions:
         Area  Area_SqFt
30  460 sqyrd      460.0
31  412 sqyrd      412.0
32  3187 sqft     3187.0
33  4608 sqft     4608.0
34  2327 sqft     2327.0
35  3445 sqft     3445.0
36  1795 sqft     1795.0
37  1458 sqft     1458.0
38  3584 sqft     3584.0
39  2491 sqft     2491.0


## 🏠 Step 4: Standardize BHK Column

In [7]:
def standardize_bhk(bhk_str):
    """Extract numeric BHK value"""
    if pd.isna(bhk_str) or bhk_str == 'N/A':
        return np.nan
    
    bhk_str = str(bhk_str).upper()
    
    # Extract first number
    numbers = re.findall(r'\d+', bhk_str)
    if numbers:
        bhk_val = int(numbers[0])
        # Validate reasonable range
        return bhk_val if 1 <= bhk_val <= 10 else np.nan
    
    return np.nan

# Apply standardization
df['BHK_Numeric'] = df['BHK'].apply(standardize_bhk)

print("🏠 BHK Standardization Results:")
print(f"   Valid BHK values: {df['BHK_Numeric'].notna().sum()}")
print(f"   Missing/Invalid: {df['BHK_Numeric'].isna().sum()}")
print(f"\n📊 BHK Distribution:")
print(df['BHK_Numeric'].value_counts().sort_index())

# Sample conversions
print("\n🔍 Sample BHK Conversions:")
print(df[['BHK', 'BHK_Numeric']].dropna().head(10))

🏠 BHK Standardization Results:
   Valid BHK values: 2790
   Missing/Invalid: 15

📊 BHK Distribution:
BHK_Numeric
1.0     302
2.0    1106
3.0    1057
4.0     277
5.0      41
6.0       5
7.0       1
8.0       1
Name: count, dtype: int64

🔍 Sample BHK Conversions:
     BHK  BHK_Numeric
0  5 BHK          5.0
1  4 BHK          4.0
2  3 BHK          3.0
3  5 BHK          5.0
4  2 BHK          2.0
5  4 BHK          4.0
6  3 BHK          3.0
7  3 BHK          3.0
8  4 BHK          4.0
9  3 BHK          3.0


## 🚿 Step 5: Clean Bathrooms Column

In [8]:
def clean_bathrooms(bath_str):
    """Extract numeric bathroom count"""
    if pd.isna(bath_str) or bath_str == 'N/A':
        return np.nan
    
    bath_str = str(bath_str)
    numbers = re.findall(r'\d+', bath_str)
    
    if numbers:
        bath_val = int(numbers[0])
        return bath_val if 1 <= bath_val <= 10 else np.nan
    
    return np.nan

# Apply cleaning
df['Bathrooms_Numeric'] = df['Bathrooms'].apply(clean_bathrooms)

print("🚿 Bathroom Cleaning Results:")
print(f"   Valid bathroom counts: {df['Bathrooms_Numeric'].notna().sum()}")
print(f"   Missing/Invalid: {df['Bathrooms_Numeric'].isna().sum()}")
print(f"\n📊 Bathroom Distribution:")
print(df['Bathrooms_Numeric'].value_counts().sort_index())

🚿 Bathroom Cleaning Results:
   Valid bathroom counts: 74
   Missing/Invalid: 2731

📊 Bathroom Distribution:
Bathrooms_Numeric
1.0     8
2.0    31
3.0    24
4.0     8
5.0     3
Name: count, dtype: int64


## 🪑 Step 6: Standardize Furnishing Status

In [9]:
def standardize_furnishing(furn_str):
    """Standardize furnishing status"""
    if pd.isna(furn_str) or furn_str == 'N/A':
        return 'Unknown'
    
    furn_str = str(furn_str).lower()
    
    if 'unfurnished' in furn_str or 'un-furnished' in furn_str:
        return 'Unfurnished'
    elif 'semi' in furn_str or 'semi-furnished' in furn_str:
        return 'Semi-Furnished'
    elif 'furnished' in furn_str or 'fully' in furn_str:
        return 'Furnished'
    else:
        return 'Unknown'

# Apply standardization
df['Furnishing_Status'] = df['Furnishing'].apply(standardize_furnishing)

print("🪑 Furnishing Standardization Results:")
print(df['Furnishing_Status'].value_counts())

🪑 Furnishing Standardization Results:
Furnishing_Status
Unfurnished       1131
Furnished          845
Semi-Furnished     776
Unknown             53
Name: count, dtype: int64


## 👤 Step 7: Standardize Seller Type

In [10]:
def standardize_seller(seller_str):
    """Standardize seller type"""
    if pd.isna(seller_str) or seller_str == 'N/A':
        return 'Unknown'
    
    seller_str = str(seller_str).lower()
    
    if 'owner' in seller_str:
        return 'Owner'
    elif 'agent' in seller_str or 'dealer' in seller_str or 'broker' in seller_str:
        return 'Agent'
    elif 'builder' in seller_str or 'developer' in seller_str:
        return 'Builder'
    else:
        return 'Unknown'

# Apply standardization
df['Seller_Type'] = df['Seller Type'].apply(standardize_seller)

print("👤 Seller Type Standardization Results:")
print(df['Seller_Type'].value_counts())

👤 Seller Type Standardization Results:
Seller_Type
Owner      2540
Agent       243
Builder      20
Unknown       2
Name: count, dtype: int64


## 🗺️ Step 8: Clean Locality Names

In [11]:
# Clean locality names
df['Locality_Clean'] = df['Locality'].fillna('Unknown')
df['Locality_Clean'] = df['Locality_Clean'].str.strip()
df['Locality_Clean'] = df['Locality_Clean'].str.title()

print("🗺️ Locality Cleaning Results:")
print(f"   Total unique localities: {df['Locality_Clean'].nunique()}")
print(f"\n📍 Top 15 Localities by Property Count:")
print(df['Locality_Clean'].value_counts().head(15))

🗺️ Locality Cleaning Results:
   Total unique localities: 1527

📍 Top 15 Localities by Property Count:
Locality_Clean
Ahmedabad                        167
Shela, Ahmedabad                  22
Nikol, Ahmedabad                  21
Gota, Ahmedabad                   21
Chandkheda, Ahmedabad             21
Vastral, Ahmedabad                19
Maninagar, Ahmedabad              18
Vaishnodevi Circle, Ahmedabad     14
Paldi, Ahmedabad                  14
South Bopal, Bopal, Ahmedabad     14
Bopal, Ahmedabad                  13
Shahibag, Ahmedabad               13
Navrangpura, Ahmedabad            13
Ambawadi, Ahmedabad               13
Motera, Ahmedabad                 12
Name: count, dtype: int64


## ⚠️ Step 9: Remove Outliers

In [12]:
# Remove outliers using IQR method
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.01)
    Q3 = df[column].quantile(0.99)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    before = len(df)
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    after = len(df_filtered)
    
    print(f"   {column}: Removed {before - after} outliers ({((before-after)/before*100):.2f}%)")
    return df_filtered

print("⚠️ Removing Outliers:")
df_clean = df.copy()

# Remove outliers from numeric columns
for col in ['Price_Lakhs', 'Area_SqFt']:
    if df_clean[col].notna().sum() > 0:
        df_clean = remove_outliers(df_clean, col)

print(f"\n✅ Final dataset size: {len(df_clean)} records")

⚠️ Removing Outliers:
   Price_Lakhs: Removed 4 outliers (0.14%)
   Area_SqFt: Removed 30 outliers (1.07%)

✅ Final dataset size: 2771 records


## 🔍 Step 10: Handle Missing Values

In [13]:
# Fill missing BHK based on area
def impute_bhk_from_area(row):
    if pd.notna(row['BHK_Numeric']):
        return row['BHK_Numeric']
    if pd.notna(row['Area_SqFt']):
        area = row['Area_SqFt']
        if area < 700:
            return 1
        elif area < 1200:
            return 2
        elif area < 1800:
            return 3
        elif area < 2500:
            return 4
        else:
            return 5
    return np.nan

df_clean['BHK_Final'] = df_clean.apply(impute_bhk_from_area, axis=1)

# Fill missing bathrooms based on BHK
def impute_bathrooms(row):
    if pd.notna(row['Bathrooms_Numeric']):
        return row['Bathrooms_Numeric']
    if pd.notna(row['BHK_Final']):
        return min(int(row['BHK_Final']), 3)  # Typically BHK count or max 3
    return np.nan

df_clean['Bathrooms_Final'] = df_clean.apply(impute_bathrooms, axis=1)

print("🔍 Missing Value Imputation Results:")
print(f"   BHK - Before: {df_clean['BHK_Numeric'].isna().sum()}, After: {df_clean['BHK_Final'].isna().sum()}")
print(f"   Bathrooms - Before: {df_clean['Bathrooms_Numeric'].isna().sum()}, After: {df_clean['Bathrooms_Final'].isna().sum()}")

🔍 Missing Value Imputation Results:
   BHK - Before: 13, After: 0
   Bathrooms - Before: 2697, After: 0


## 📊 Step 11: Final Data Quality Check

In [14]:
# Remove rows with critical missing values
critical_columns = ['Price_Lakhs', 'Area_SqFt', 'BHK_Final', 'Locality_Clean']
df_final = df_clean.dropna(subset=critical_columns)

print("="*60)
print("FINAL CLEANED DATASET SUMMARY")
print("="*60)
print(f"\n📊 Dataset Shape: {df_final.shape}")
print(f"\n✅ Records retained: {len(df_final)} out of {len(df_raw)} ({(len(df_final)/len(df_raw)*100):.1f}%)")

print("\n📋 Missing Values in Final Dataset:")
missing_final = df_final[['Price_Lakhs', 'Area_SqFt', 'BHK_Final', 'Bathrooms_Final', 
                          'Furnishing_Status', 'Seller_Type', 'Locality_Clean']].isnull().sum()
print(missing_final)

print("\n📈 Final Data Statistics:")
print(df_final[['Price_Lakhs', 'Area_SqFt', 'BHK_Final', 'Bathrooms_Final']].describe())

FINAL CLEANED DATASET SUMMARY

📊 Dataset Shape: (2771, 20)

✅ Records retained: 2771 out of 2989 (92.7%)

📋 Missing Values in Final Dataset:
Price_Lakhs          0
Area_SqFt            0
BHK_Final            0
Bathrooms_Final      0
Furnishing_Status    0
Seller_Type          0
Locality_Clean       0
dtype: int64

📈 Final Data Statistics:
       Price_Lakhs     Area_SqFt    BHK_Final  Bathrooms_Final
count  2771.000000   2771.000000  2771.000000      2771.000000
mean   4127.922772   1557.564778     2.513533         2.383255
std    3030.539332   1856.256222     0.888401         0.686377
min     100.000000     11.000000     1.000000         1.000000
25%     400.000000    630.000000     2.000000         2.000000
50%    4500.000000   1050.000000     2.000000         2.000000
75%    6500.000000   1640.000000     3.000000         3.000000
max    9990.000000  10000.000000     8.000000         5.000000


## 💾 Step 12: Export Cleaned Dataset

In [15]:
# Select final columns
columns_to_keep = [
    'Property Title',
    'Price_Lakhs',
    'Area_SqFt',
    'BHK_Final',
    'Bathrooms_Final',
    'Furnishing_Status',
    'Seller_Type',
    'Locality_Clean',
    'City',
    'Source'
]

df_export = df_final[columns_to_keep].copy()

# Rename for clarity
df_export.columns = [
    'Property_Title',
    'Price_Lakhs',
    'Area_SqFt',
    'BHK',
    'Bathrooms',
    'Furnishing',
    'Seller_Type',
    'Locality',
    'City',
    'Source'
]

# Export to CSV
output_file = 'cleaned_real_estate_data.csv'
df_export.to_csv(output_file, index=False)

print(f"\n💾 Cleaned dataset saved: {output_file}")
print(f"   Shape: {df_export.shape}")
print(f"\n✅ DATA CLEANING COMPLETED SUCCESSFULLY!")

# Preview
print("\n👀 Preview of Cleaned Data:")
df_export.head(10)


💾 Cleaned dataset saved: cleaned_real_estate_data.csv
   Shape: (2771, 10)

✅ DATA CLEANING COMPLETED SUCCESSFULLY!

👀 Preview of Cleaned Data:


,Property_Title,Price_Lakhs,Area_SqFt,BHK,Bathrooms,Furnishing,Seller_Type,Locality,City,Source
30,"4 BHK Flat for Sale in Aristo Anantam, Chharod...",272.0,460.0,4.0,3.0,Unfurnished,Builder,"Aristo Anantam, Chharodi, Ahmedabad",Ahmedabad,MagicBricks
31,"4 BHK Flat for Sale in Propus Clara, Vastral, ...",137.0,412.0,4.0,3.0,Unfurnished,Builder,"Propus Clara, Vastral, Ahmedabad",Ahmedabad,MagicBricks
32,"4 BHK Flat for Sale in Ashima The Sovereign, T...",434.0,3187.0,4.0,3.0,Unfurnished,Builder,"Ashima The Sovereign, Thaltej, Ahmedabad",Ahmedabad,MagicBricks
33,"4 BHK Flat for Sale in AARON ELINOR 51, Scienc...",294.0,4608.0,4.0,3.0,Unfurnished,Agent,"Aaron Elinor 51, Science City, Ahmedabad",Ahmedabad,MagicBricks
34,"4 BHK Flat for Sale in Satyamev Luxor, Ambli, ...",304.0,2327.0,4.0,3.0,Unfurnished,Agent,"Satyamev Luxor, Ambli, Ahmedabad",Ahmedabad,MagicBricks
35,"4 BHK Flat for Sale in Super Shaligram, Gota, ...",182.0,3445.0,4.0,3.0,Unfurnished,Agent,"Super Shaligram, Gota, Ahmedabad",Ahmedabad,MagicBricks
36,"3 BHK Flat for Sale in Serenity Shivam, South ...",7180.0,1795.0,3.0,3.0,Unfurnished,Agent,"Serenity Shivam, South Bopal, Bopal, Ahmedabad",Ahmedabad,MagicBricks
37,2 BHK Flat for Sale in Om The Green Parmeshwar...,6800.0,1458.0,2.0,2.0,Unfurnished,Agent,"Om The Green Parmeshwar, Jagatpur, Ahmedabad",Ahmedabad,MagicBricks
38,"4 BHK Flat for Sale in Siddharth Vrundavan, Ja...",175.0,3584.0,4.0,3.0,Unfurnished,Agent,"Siddharth Vrundavan, Jagatpur, Ahmedabad",Ahmedabad,MagicBricks
39,4 BHK Flat for Sale in A Shridhar Anantara Imp...,245.0,2491.0,4.0,3.0,Unfurnished,Agent,"A Shridhar Anantara Imperial, Science City, Ah...",Ahmedabad,MagicBricks
